In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # modify root dispatcher

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
dispatcher

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher = instrument.get_dispatcher("llama_index.core.base.query_engine")

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.query_engine', event_handlers=[], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x137796830>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher.root

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_result = query_engine.query("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent
QueryEndEvent


In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='VectorIndexRetriever._retrieve-9a7ed109-2dfc-4f95-b40a-bd2a860bbebb', parent_id='BaseRetriever.retrieve-c7e5911f-8d8f-491f-8587-bd24328b314c', start_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 114256), end_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 259116), duration=0.14486),
 SimpleSpan(id_='BaseRetriever.retrieve-c7e5911f-8d8f-491f-8587-bd24328b314c', parent_id='RetrieverQueryEngine._query-516036eb-9a85-4aa3-a2da-6956db99cd53', start_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 114017), end_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 259339), duration=0.145322),
 SimpleSpan(id_='LLM.predict-30dbcb1a-85d5-4ac7-856a-2d0ab80dec94', parent_id='Refine.get_response-db0d21ba-a909-4cff-91d0-b5687f5a7525', start_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 262980), end_time=datetime.datetime(2024, 3, 7, 3, 7, 48, 831717), duration=1.568737),
 SimpleSpan(id_='Refine.get_response-db0d21ba-a909-4cff-91d0-b5687f5a7525', parent_id='CompactAndRefine.get_response-c9d6cc90-cba5-4

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent
QueryEndEvent


In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='VectorIndexRetriever._retrieve-9a7ed109-2dfc-4f95-b40a-bd2a860bbebb', parent_id='BaseRetriever.retrieve-c7e5911f-8d8f-491f-8587-bd24328b314c', start_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 114256), end_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 259116), duration=0.14486),
 SimpleSpan(id_='BaseRetriever.retrieve-c7e5911f-8d8f-491f-8587-bd24328b314c', parent_id='RetrieverQueryEngine._query-516036eb-9a85-4aa3-a2da-6956db99cd53', start_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 114017), end_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 259339), duration=0.145322),
 SimpleSpan(id_='LLM.predict-30dbcb1a-85d5-4ac7-856a-2d0ab80dec94', parent_id='Refine.get_response-db0d21ba-a909-4cff-91d0-b5687f5a7525', start_time=datetime.datetime(2024, 3, 7, 3, 7, 47, 262980), end_time=datetime.datetime(2024, 3, 7, 3, 7, 48, 831717), duration=1.568737),
 SimpleSpan(id_='Refine.get_response-db0d21ba-a909-4cff-91d0-b5687f5a7525', parent_id='CompactAndRefine.get_response-c9d6cc90-cba5-4

### Very Primitive Tree Viz

In [ ]:
%pip install treelib -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from treelib import Node, Tree

tree = Tree()

In [ ]:
for span in reversed(dispatcher.span_handler.completed_spans[:8]):
    tree.create_node(
        tag=f"{span.id_} ({span.duration})",
        identifier=span.id_,
        parent=span.parent_id,
        data=span.duration,
    )

In [ ]:
print(tree.show(stdout=False))

BaseQueryEngine.query-45040750-4cf2-4190-a772-41571ea6d82e (1.719234)
└── RetrieverQueryEngine._query-516036eb-9a85-4aa3-a2da-6956db99cd53 (1.718296)
    ├── BaseRetriever.retrieve-c7e5911f-8d8f-491f-8587-bd24328b314c (0.145322)
    │   └── VectorIndexRetriever._retrieve-9a7ed109-2dfc-4f95-b40a-bd2a860bbebb (0.14486)
    └── BaseSynthesizer.synthesize-74f37a74-a288-4ed5-8e3e-b8999970866a (1.572873)
        └── CompactAndRefine.get_response-c9d6cc90-cba5-44b9-b053-e49763222ae4 (1.572551)
            └── Refine.get_response-db0d21ba-a909-4cff-91d0-b5687f5a7525 (1.570478)
                └── LLM.predict-30dbcb1a-85d5-4ac7-856a-2d0ab80dec94 (1.568737)

